# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from bert_serving.client import BertClient
bc = BertClient()
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

C:\Users\Savannah\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Load tag data

In [2]:
# Load tag data
column = "space" # Set to "space" or "nospace"
transcriber_tags = pd.read_excel("../data/tags.xlsx", converters={column:str}, sheet_name="ZooniverseTags")
expert_tags = pd.read_excel("../data/tags.xlsx", converters={column:str}, sheet_name="ExpertTags")

# Print (some) tags that Word2Vec does not understand.

Many of these tags are short phrases which Word2Vec does not have in its base vocabulary. I do not currently have the resources to fine-tune Word2Vec to add these phrases. For this reason, the current version of this notebook only compares BERT tag embeddings.

In [3]:
for e in expert_tags[column]:
    try:
        wv[e]
    except Exception as err:
        print(err)

"word '1 in 10 rations' not in vocabulary"
"word 'afi courses' not in vocabulary"
"word 'army films' not in vocabulary"
"word 'army insurance' not in vocabulary"
"word 'army living' not in vocabulary"
"word 'army scorecard plan' not in vocabulary"
"word 'army talks' not in vocabulary"
"word 'attitude toward england' not in vocabulary"
"word 'attitude toward france' not in vocabulary"
"word 'attitude towards germany' not in vocabulary"
"word 'attitude toward italy' not in vocabulary"
"word 'base newspapers' not in vocabulary"
"word 'base units' not in vocabulary"
"word 'branch of service' not in vocabulary"
"word 'british papers' not in vocabulary"
"word 'business ownership' not in vocabulary"
"word 'cbi roundup' not in vocabulary"
"word 'chemical warfare' not in vocabulary"
"word 'civilian skills' not in vocabulary"
"word 'combat flying' not in vocabulary"
"word 'combat stress' not in vocabulary"
"word 'confidence in ship' not in vocabulary"
"word 'confidence in troops' not in vocabula

# Determine best approximations for transcriber tags

## Get embeddings

In [4]:
# Redefine encoding function to allow progress tracking
count = 0
def encode(tag):
    global count
    if count%500 == 0:
        print(count)
    count += 1
    return bc.encode([tag])[0]

# Get embeddings
expert_embed = [encode(i) for i in expert_tags[column]]
transcriber_embed = [encode(i) for i in transcriber_tags[column]]

0
500
1000
1500
2000
2500
3000
3500
4000


## Compare embeddings

In [5]:
# Compare transcriber tag at index i to all expert tags
def compare_all(i):
    
    # Track best results
    result = {
        "index": i,
        "transcriber_tag": transcriber_tags['nospace'].iloc[i],
        "expert_tag_1": None,
        "score_1": -1,
        "expert_tag_2": None,
        "score_2": -1,
        "expert_tag_3": None,
        "score_3": -1
    }
    
    # Make comparisons
    for j in range(len(expert_tags)):
        score = np.dot(transcriber_embed[i], expert_embed[j]) / \
            (np.linalg.norm(transcriber_embed[i]) * np.linalg.norm(expert_embed[j]))
        if score > result['score_1']:
            result['expert_tag_1'] = expert_tags[column].iloc[j]
            result['score_1'] = score
        elif score > result['score_2']:
            result['expert_tag_2'] = expert_tags[column].iloc[j]
            result['score_2'] = score
        elif score > result['score_3']:
            result['expert_tag_3'] = expert_tags[column].iloc[j]
            result['score_3'] = score
            
    return result

results = [compare_all(i) for i in range(len(transcriber_tags))]

# Write results as CSV

In [6]:
# Convert list of results to dataframe
df = pd.DataFrame(results)
# Sort results by cosine similarity
df = df.sort_values('score_1', ascending=False)
# Write to excel file
df.to_csv("../data/tag_match_BAS.csv")

# Cluster words that lack a close-enough approximation

In [8]:
# Perform KMeans on embeddings with low similarity scores
k = 30 # Adjust number of clusters

cluster_results = [results[i] for i in range(len(results))
                   if results[i]['score_1'] < 0.75 ] # Adjust similarity threshold

cluster_embeds = [transcriber_embed[cluster_results[i]['index']] 
                  for i in range(len(cluster_results))]

kmeans = KMeans(n_clusters=k).fit(cluster_embeds)

for i in range(k):
    print("Cluster", i)
    print([cluster_results[j]['transcriber_tag'] 
           for j in range(len(kmeans.labels_))
           if kmeans.labels_[j] == i])

Cluster 0
['africanamerican', 'africanamericans', 'africanamericansoldier', 'africanamericansoldiers', 'antisemitism', 'desegregation', 'evalgelism', 'fascism', 'hitler', 'laborstrikes', 'lynching', 'nazism', 'race-relations', 'racial-discrimination', 'sexism', 'sexist']
Cluster 1
['allforone', 'b-24', 'b-29', 'coloredandwhite', 'columnacolumnb', 'goodpoints', 'northandsouth', 'northvssouth', 'oneanddone', 'oneandone', 'oneteam', 'youngvsold']
Cluster 2
['amallbusiness', 'biggovernmentisbad', 'callingsomeoneout', 'entepreneur', 'gethimoutofbarracks', 'ishouldnthavetofightformyrights', 'iwanttotravelmoreoften', 'keepthepoliticsindc', 'makemeanofficer', 'officersneedtotryit', 'onceasoldieralwaysasoldier', 'privileges-for-officers', 'proudtoserve', 'servingthecountry', 'settling_down', 'sickofwar', 'soldiersdontseecolor', 'stretchingthesamesentencetoreachwordlimi', 'too_old_to_fight', 'wecanallgetalong', 'whenwillthewarbeover', 'heknowsexactlywhathewantstodo']
Cluster 3
['believeme', 'beq